<a href="https://colab.research.google.com/github/chijade/coursera-capstone/blob/master/Capstone_Project_The_Battle_of_Neighborhoods_Solution(Week_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

In [37]:
CLIENT_ID = 'UDDGPQBBCX3LMW4X4WBTU5BPYTVPOLEEE33X1YLBD2MGY4SI' # your Foursquare ID
CLIENT_SECRET = 'WQC2VJAXOWMYPOACCTEYZUT2K0YNBPYPJMY5IYAGXXJXAVCW' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UDDGPQBBCX3LMW4X4WBTU5BPYTVPOLEEE33X1YLBD2MGY4SI
CLIENT_SECRET:WQC2VJAXOWMYPOACCTEYZUT2K0YNBPYPJMY5IYAGXXJXAVCW


In [0]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Da Nang",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [40]:
print(data)

<Response [200]>


In [30]:
#let's try to see how our data source is going
search_query = 'coffeeshop'
radius = 500
print(search_query + ' .... OK!')

coffeeshop .... OK!


In [44]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [45]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Da Nang', 'Da Nang')

In [46]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 16.076950003892705, 'lng': 108.22982675100442},
  'sw': {'lat': 16.059727142671775, 'lng': 108.21333222890732}},
 41)

In [47]:
d["geocode"]

{'cc': 'VN',
 'center': {'lat': 16.06778, 'lng': 108.22083},
 'displayString': 'Da Nang, Thành Phố Đà Nẵng, Vietnam',
 'geometry': {'bounds': {'ne': {'lat': 16.11072, 'lng': 108.276871},
   'sw': {'lat': 15.982205, 'lng': 108.141861}}},
 'longId': '72057594039511928',
 'slug': 'turan-vietnam',
 'what': '',
 'where': 'da nang'}

In [48]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [49]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [50]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 41


{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-5-5a26a41a31ac6c676705e94c-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'primary': True,
    'shortName': 'Coffee Shop'}],
  'id': '5a26a41a31ac6c676705e94c',
  'location': {'address': '39',
   'cc': 'VN',
   'city': 'Đà Nẵng',
   'country': 'Việt Nam',
   'crossStreet': 'Nguyễn Thái Học',
   'formattedAddress': ['39 (Nguyễn Thái Học)',
    'Đà Nẵng',
    'Thành Phố Đà Nẵng',
    'Việt Nam'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 16.068062879455486,
     'lng': 108.22351222071603}],
   'lat': 16.068062879455486,
   'lng': 108.22351222071603,
   'postalCode': '551105',
   'state': 'Thành Phố Đà Nẵng'},
  'name': 'cộng cà ph

In [60]:
items[1]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-5-4d4361d01928a35daba1ad70-1',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'primary': True,
    'shortName': 'Coffee Shop'}],
  'id': '4d4361d01928a35daba1ad70',
  'location': {'address': '17 Phan Đình Phùng',
   'cc': 'VN',
   'city': 'Hải Châu',
   'country': 'Việt Nam',
   'formattedAddress': ['17 Phan Đình Phùng',
    'Hải Châu',
    'Thành Phố Đà Nẵng',
    'Việt Nam'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 16.070069923328905,
     'lng': 108.2234060288238}],
   'lat': 16.070069923328905,
   'lng': 108.2234060288238,
   'state': 'Thành Phố Đà Nẵng'},
  'name': 'Phố Xưa',
  'photos': {'count': 0, 'groups': []}}}

In [72]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': '39', 'crossStreet': 'Nguyễn Thái Học', 'lat': 16.068062879455486, 'lng': 108.22351222071603, 'labeledLatLngs': [{'label': 'display', 'lat': 16.068062879455486, 'lng': 108.22351222071603}], 'postalCode': '551105', 'cc': 'VN', 'city': 'Đà Nẵng', 'state': 'Thành Phố Đà Nẵng', 'country': 'Việt Nam', 'formattedAddress': ['39 (Nguyễn Thái Học)', 'Đà Nẵng', 'Thành Phố Đà Nẵng', 'Việt Nam']}
{'address': '17 Phan Đình Phùng', 'lat': 16.070069923328905, 'lng': 108.2234060288238, 'labeledLatLngs': [{'label': 'display', 'lat': 16.070069923328905, 'lng': 108.2234060288238}], 'cc': 'VN', 'city': 'Hải Châu', 'state': 'Thành Phố Đà Nẵng', 'country': 'Việt Nam', 'formattedAddress': ['17 Phan Đình Phùng', 'Hải Châu', 'Thành Phố Đà Nẵng', 'Việt Nam']}
{'address': '37-39 Pasteur', 'lat': 16.069698707615007, 'lng': 108.21804495108044, 'labeledLatLngs': [{'label': 'display', 'lat': 16.069698707615007, 'lng': 108.21804495108044}], 'cc': 'VN', 'city': 'Đà Nẵng', 'state': 'Thành Phố Đà Nẵng', 'cou

,uid,name,shortname,address,postalcode,lat,lng
0,5a26a41a31ac6c676705e94c,cộng cà phê,Coffee Shop,,551105,16.068063,108.223512
1,5627c637498e21ca8f40f4be,Leaf Drinks and More,Coffee Shop,,550000,16.067020,108.224790
2,55ff7cf0498e203b2932d5fd,Namunamu Cafe 180,Café,,550000,16.066268,108.224496
3,51bd1671498e58791c2ea92d,Freeman Coffee,Coffee Shop,,550000,16.070851,108.221230
4,4c25fdd8f1272d7f33ac85c5,Highlands Coffee,Café,,60999,16.069946,108.224961


As we can see that there are many coffee shops without address we need to execute hasattr() to determine if each object (coffeeshop) has a contribute
(address)


In [74]:
DaNang_center = d["geocode"]["center"]
DaNang_center

{'lat': 16.06778, 'lng': 108.22083}

In [77]:
from folium import plugins

# create map of Helsinki using latitude and longitude values
map_DaNang = folium.Map(location=[16.06778, 108.22083], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_DaNang)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_DaNang.add_child(plugins.HeatMap(hm_data))

map_DaNang

In [79]:
lat = 16.06778
lng = 108.22083
map_DaNang = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="O Day Roi!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_DaNang)
map_DaNang